In [5]:
import pandas as pd
import numpy as np
import os
import time
from PIL import Image
import requests
from io import BytesIO
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="disn-wildlife")
import datamart_geo
geo_data = datamart_geo.GeoData.download() 
import uuid
from statistics import mean
from PIL import Image
from numpy import asarray


In [6]:
from pathlib import Path

disk_dir = Path("data/disk/")
lmdb_dir = Path("data/lmdb/")
hdf5_dir = Path("data/hdf5/")

# disk_dir.mkdir(parents=True, exist_ok=True)
# lmdb_dir.mkdir(parents=True, exist_ok=True)
# hdf5_dir.mkdir(parents=True, exist_ok=True)

In [7]:
def read_many_disk(num_images):
    """ Reads image from disk.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []

    # Loop over all IDs and read each image in one by one
    for image_id in range(num_images):
        images.append(np.array(Image.open(disk_dir / f"{image_id}.png")))

    with open(disk_dir / f"{num_images}.csv", "r") as csvfile:
        reader = csv.reader(
            csvfile, delimiter=" ", quotechar="|", quoting=csv.QUOTE_MINIMAL
        )
        for row in reader:
            labels.append(row[0])
    return images, labels

def store_many_disk(images, img_id):
    """ Stores an array of images to disk
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    # Save all the images one by one
    for i, image in zip(img_id, images):
        Image.fromarray(image).save(disk_dir / f"{i}.png")

    # Save all the labels to the csv file
    with open(disk_dir / f"{num_images}.csv", "w") as csvfile:
        writer = csv.writer(
            csvfile, delimiter=" ", quotechar="|", quoting=csv.QUOTE_MINIMAL
        )
        for ids in img_id:
            # This typically would be more than just one value per row
            writer.writerow([ids])

In [8]:
# def reverse_geocoding(lat, long):
#     point = Point(lat, lon)
#     try:
#         location = locator.reverse(point)
#         adress = location.raw['address']
#         return address.get("state"), address.get("country")
#     except (AttributeError, KeyError, ValueError):
#         return None, None

In [ ]:
# path = "extraction/data_april12-13/"
# list_path = os.listdir(path=path)
# list_path.remove('.ipynb_checkpoints')
# list_path
# dfs=[]
# urls=[]
# labels = []
# for file in list_path:
#     data = pd.read_parquet("extraction/data_april12-13/"+file)
#     data["location"]=np.where(data["location"]== "None", None, data["location"])
#     data["location"]=np.where(data["location"]== "US", "USA", data["location"])
#     data["location"]=np.where(data["location"]== "GB", "United Kingdom", data["location"])
#     # df = data[(data["final_label"] == "an animal body part") | (data["final_label"] == "a real animal")]
#     df= data
#     df["loc"] = df["location"].apply(lambda x: geo_data.resolve_name(x) if x else None)
#     # time.sleep(1)
#     # df["location"]= df["loc"].apply(lambda loc: tuple(loc.point) if loc else None)
#     df['loc_name'] = df["loc"].apply(lambda loc: loc.name if loc else None)
#     df['lat'] = df["loc"].apply(lambda loc: loc.latitude if loc else None)
#     df['lon'] = df["loc"].apply(lambda loc: loc.longitude if loc else None)
#     df['country'] = df["loc"].apply(lambda loc: loc.get_parent_area(level=0).name if loc else None)
#     # df[["state", "country"]] = df.apply(lambda x: reverse_geocoding(x["lat"], x["lon"]) if loc else None)
#     df = df.drop(columns=["loc"])
#     dfs.append(df)
# df = pd.concat(dfs)
# df = df.reset_index(drop=True)
# df["id"] = df.apply(lambda _: uuid.uuid4(), axis=1)

In [8]:
df = pd.read_csv("extraction/complete_days_12_13.csv")

In [15]:
len(df)

21903

In [14]:
images = []
ids= []
count=0
for _, row in df.iterrows():
    try:
        response = requests.get(row["image"])
        img = Image.open(BytesIO(response.content))
        images.append(asarray(img))
        labels.append(row["final_label"])
        ids.append(row["id"])
        count +=1
        if count == 1000:
            print("appended 1000")
            count = 0
            store_many_disk(images, ids)
            images = []
            ids= []
    except Exception:
        continue

appended 1000
appended 1000
appended 1000


In [15]:
len(images)

3349

In [ ]:
images = []
ids= []
labels = []

count=0
for _, row in df[15000:].iterrows():
    try:
        response = requests.get(row["image"])
        img = Image.open(BytesIO(response.content))
        images.append(asarray(img))
        labels.append(row["final_label"])
        ids.append(row["id"])
        count +=1
        
        
        if count == 1000:
            print("appended 1000")
            count = 0
            store_many_disk(images, ids)
            images = []
            ids= []
            labels = []
    except Exception:
        continue

appended 1000
appended 1000


In [ ]:
store_many_disk(images, ids)